# Desafio

A proposta desse desafio é usar o Algoritmo de Shor para decriptar uma mensagem encriptada pelo método RSA. Para realizar esta tarefa, será necessário encontrar fatores primos, $p$ e $q$, de um dado número inteiro, $N$, e, \underline{sem} seguida, com o uso das chaves pública, $e$, e privada, $d$, será possível descobrir a mensagem cifrada. 

### Relembrando os passos do método RSA para encriptação

1. Selecionar um par de números primos, $p$ e $q$ e calcular N, tal que  

$$p \times q = N$$

2. Calcular $\phi$:

$$\phi = (p-1)(q-1)$$

3. Cálculo das chaves pública, $e$, e privada, $d$, que devem satisfazer as seguintes condições:

$$\text{mdc}(e, p-1) = \text{mdc}(e, q-1) = 1$$
$$e \times d \mod \phi = 1$$


### Passos para decifrar uma mensagem

Uma vez que temos $N$, $e$ e $d$, sujeitos às restrições dadas acima, estamos prontos para encriptar e decriptar mensagens. Sabendo que as mensagens devem ser convertidas em números inteiros no protocolo RSA, uma mensagem $m$ pode ser encriptada da seguinte maneira:

$$m^* = m^e\mod N$$

Uma vez que a mensagem é encriptada, qualquer pessoa com acesso a chave privada, $d$, pode decifrar a mensagem através da seguinte relação:

$$m' = m^{*d} \mod N$$

Logo, $m = m'$ caso a mensagem tenha sido decrifrada com sucesso.




Colocar um exemplo com o número 15.

In [12]:
from qat.lang.AQASM import *
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import PyLinalg
from qat.qpus import get_default_qpu
import numpy as np
from math import modf, gcd
import matplotlib.pyplot as plt

# Algoritmo de Shor

Neste notebook, vamos implementar um algoritmo quântico para fatorar o número 15. Escreveremos um código de implementação do Algoritmo de Shor em `myQLM`, seguindo o material apresentado em sala.

Material de apoio:
- **Algoritmo de Shor**: aula shor
- **Estimativa de Fase Quântica**: aula qpe
- **Transformada de Fourier Quântica**: aula qft

## Revisão do Algoritmo de Shor
Lembrando das aulas anteriores, o Algoritmo de Shor pode ser utilizado para encontrar fatores de um número inteiro $N$, sendo

$$N = pq$$

onde $p$ e $q$ são números primos. Isso é feito em quatro etapas principais, semelhantes à implementação da estimativa de fase quântica. Neste caso, usaremos dois registradores de qubits. O primeiro registro terá $n = \lfloor \log_2 N^2 \rfloor = \lfloor 2 \log_2 N \rfloor$ qubits e conterá os qubits de medição. O segundo registrador terá $m = \lceil \log_2 N \rceil$ qubits, e será o autoestado para a estimativa da fase quântica.

1. Primeiro, começamos inicializando nossos qubits. Criamos uma superposição de todos os estados de base computacional $2^n$ nos qubits de medição $n$ aplicando uma porta Hadamard ($H$) em cada qubit começando no estado $\vert0\rangle^{\otimes n}$ . Também inicializamos os qubits de destino $m = \lceil \log_2 N \rceil$  no estado $\vert1\rangle$. Aqui, o expoente $\otimes n$ significa que temos um produto tensorial dos estados de $n = 2m$ qubits.

2. Em segundo lugar, aplicamos o operador unitário $U|x\rangle = |ax \mod N\rangle $ com várias potências nos qubits alvo, controlando-o com cada um dos diferentes qubits de medição. O operador unitário, neste caso, implementa a exponenciação modular. O esquema abaixo mostra a ordenação e respectivas potências.



<img src="shorimage.png" width=500 />



3. Em terceiro lugar, aplicamos uma transformada de Fourier quântica inversa nos qubits de medição $n$.

4. Finalmente, medimos os primeiros $n$ qubits, colapsando para o estado $|\ell\rangle$ .

Depois que os resultados da medição forem determinados, precisaremos fazer um pós-processamento clássico adicional para determinar os fatores ou decidir executar o programa novamente.

5. A expansão em uma fração continuada de $\frac{\ell}{2^m}$, onde $\ell$ é a saída do algoritmo, é dada por
$$\frac{\ell}{2^m}=\cfrac{1}{a_1 + \cfrac{1}{a_2 + \cfrac{1}{ \ddots + \cfrac{1}{a_k} }}}$$
Os sucessivos convergentes da fração continuada são $[a_1]$, $[a_1,a_2]$, $[a_1,a_2,a_3]$ e assim por diante onde
$$[a_1]=\cfrac{1}{a_1},\;\;[a_1,a_2]=\cfrac{1}{a_1 + \cfrac{1}{a_2 }},\;\;[a_1,a_2,a_3]=\cfrac{1}{a_1 + \cfrac{1}{a_2 + \cfrac{1}{ a_3}}}, \;\;\cdots$$
O último convergente é $[a_1,a_2,...,a_k]$, que é igual a $\frac{\ell}{2^m}$. No algoritmo de Shor, temos que escolher o convergente mais próximo de $\frac{\ell}{2^m}$ tal que o denominador seja menor do que $N$.


## Implementando o Algoritmo de Shor com o `myQLM`

Neste notebook, vamos implementar o Algoritmo de Shor para fatorar o número 35.


### 1. Inicializando os qubits

Inicializamos o registrador de medida como uma superposição uniforme através da operação $H^{\otimes n}|0\rangle^{\otimes n}$. O segundo registrador, onde será aplicada a exponenciação modular, é inicializado no estado $|1\rangle \equiv |00...001\rangle$.

Logo, o estado inicializado, $|\psi_0\rangle$, é dado por

$$|\psi_0\rangle = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}|i\rangle\otimes |1\rangle$$

#### 1.1 Código de inicialização

In [13]:
import math
math.log(35**2)

7.110696122978827

In [74]:
#n=4; m=5; num_qubits = n+m
#a=8; N=35 #usar 

a=8; m=5; n=4
num_qubits = n+m
N=35

def initialize_qubits():
    init_circ = QRoutine()
    wires = init_circ.new_wires(num_qubits)

    for i in range(n):
        init_circ.apply(H, wires[i])

    init_circ.apply(X, wires[num_qubits - 1])

    return init_circ

### 2. Exponenciação Modular Quântica

Nessa etapa, o segundo registrador deve armazenar, em superposição, os valores da exponenciação modular, $a^{x}\mod N$, para cada inteiro $x$. Portanto, devemos criar um operador $M$, que aplicado à $|\psi_0\rangle$, resulta em

$$ |\psi_1\rangle = M|\psi_0\rangle = \frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1}|x\rangle\otimes |a^{x}\mod N\rangle $$

Uma maneira simples de implementar a exponenciação modular é através de uma estrutura muito semelhante à Estimativa de Fase Quântica (QPE). 

### Multiplicação modular

Circuito para realizar a multiplicação modular $|8y\mod 35\rangle$

<img src="mult_modular.png" width=250 />

In [75]:
def modular_exponentiation():
    #"""
    circ = QRoutine()
    wires = circ.new_wires(num_qubits)

    # |9^1 mod 35> = |01001>
    circ.apply(CNOT, n-1, 11)

    # |9^2 mod 35> = |01011>
    circ.apply(CNOT, n-2, 11)
    circ.apply(CNOT, n-2, 13)

    # |9^3 mod 35> = |11101>
    circ.apply(CNOT, n-3, 10)
    circ.apply(CNOT, n-3, 11)
    circ.apply(CNOT, n-3, 12)

    # |9^4 mod 35> = |10000>
    circ.apply(CNOT, n-4, 10)
    circ.apply(CNOT, n-4, 14)

    # |9^5 mod 35> = |00100>
    circ.apply(CNOT, n-5, 12)
    circ.apply(CNOT, n-5, 14)
    #"""

    """
    circ = QRoutine()
    wires = circ.new_wires(num_qubits)
    # |aaaa>|00001>
    # |8^2^0 mod 35> = |01000>
    circ.apply(CNOT, n-1, 5)
    circ.apply(CNOT, n-1, 8)

    # |8^2^1 mod 35> = |11101>
    circ.apply(CNOT, n-2, 4)
    circ.apply(CNOT, n-2, 5)
    circ.apply(CNOT, n-2, 6)

    # |8^2^2 mod 35> = |00001>
    """
    return circ

def controlled_modular_exponentiation(i):
    circ = QRoutine()
    wires = circ.new_wires(1 + 4) #1 qubit de controle e 4 onde ocorrem a multiplicação
    for j in range(2**i):

        circ.apply(CNOT,0,1)
        circ.apply(CNOT,0,3)
        circ.apply(CNOT,0,5)

        circ.apply(CCNOT,0,2,1)
        circ.apply(CCNOT,0,2,3)
        circ.apply(CCNOT,0,2,5)

        #SWAP
        circ.apply(CCNOT,0,2,5)
        circ.apply(CCNOT,0,5,2)
        circ.apply(CCNOT,0,2,5)

    return circ

### 3. Criando o programa

O programa consiste na aplicação das três principais operações descritas acima:

1. Inicialização do circuito
2. Exponenciação modular
3. Transformada Quântica de Fourier (QFT)

As medidas serão feitas no registrador da QFT e usadas no pós processamento a fim de se encontrar o período $r$.


In [76]:
"""
shor = Program()
qr = shor.qalloc(num_qubits)

init_circ = initialize_qubits()
modular_exp = modular_exponentiation()

shor.apply(init_circ, qr)
shor.apply(modular_exp, qr)
QFT(n)(tuple([qr[i] for i in range(n)]))

#QFT(n)(qr[0], qr[1], qr[2], qr[3])

circuit = shor.to_circ()
"""
shor = Program()
qr = shor.qalloc(num_qubits)
init_circ = initialize_qubits()
shor.apply(init_circ, qr)

for i in range(4):
    modularExponentiation = controlled_modular_exponentiation(i)
    shor.apply(modularExponentiation, qr[3-i], qr[4], qr[5], qr[6], qr[7], qr[8])


QFT(4)(qr[0], qr[1], qr[2], qr[3])
circuit = shor.to_circ()

#Obs: na interface web do jupyter é possível dar zoom na imagem e visualizar melhor o circuito
%qatdisplay circuit --svg


In [77]:
job = circuit.to_job()
result = get_default_qpu().submit(job)

states = []
for i, sample in enumerate(result):
     print("i: %s State %s: probability %s " % (i, sample.state, sample.probability))
     states.append(sample.state)

i: 0 State |000000001>: probability 0.14062499999999983 
i: 1 State |000001000>: probability 0.09765624999999986 
i: 2 State |000010100>: probability 0.09765624999999986 
i: 3 State |000101000>: probability 0.0039062499999999965 
i: 4 State |000110100>: probability 0.0039062499999999965 
i: 5 State |001000001>: probability 0.007812499999999989 
i: 6 State |001001000>: probability 0.003906249999999995 
i: 7 State |001010100>: probability 0.003906249999999995 
i: 8 State |001100001>: probability 0.007812499999999991 
i: 9 State |001101000>: probability 0.003906249999999995 
i: 10 State |001110100>: probability 0.003906249999999995 
i: 11 State |010000001>: probability 0.002288228271980094 
i: 12 State |010001000>: probability 0.0006702065439601939 
i: 13 State |010010100>: probability 0.000670206543960194 
i: 14 State |010100001>: probability 0.013336771728019886 
i: 15 State |010101000>: probability 0.02276729345603977 
i: 16 State |010110100>: probability 0.022767293456039776 
i: 17 St

In [36]:
# Converte os autoestados medidos no registrador da QFT em decimais
import string
decimals = [[str(int(b)) for b in states[i]] for i in range(len(states))]

for i,lista in enumerate(decimals):
    decimals[i] = list(reversed(lista[:n]))
    decimals[i] = int(''.join(decimals[i]),2)
    

decimals = set(decimals)
decimals

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [18]:
# Transformando os valores binários medidos no primeiro registrador em números decimais.
import string
decimals = [[str(int(b)) for b in states[i]] for i in range(len(states))]

for i,lista in enumerate(decimals):
    decimals[i] = list(reversed(lista[:n]))
    decimals[i] = int(''.join(decimals[i]),2)

decimals = set(decimals)

In [19]:
decimals

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [20]:
def frac_cont(decimal_medido):
    """
    Algoritmo de fracoes continuadas    
        Argumento: decimal_medido
        Output: Candidato a período r
    """
    M = decimal_medido/2**n
    for _ in range(2*n):
        b, r = modf(M)
        if b > round(0, 4):
            M = 1/b
        else:
            break
    return int(r)

In [72]:
from contfrac import continued_fraction,convergents  # pip install contfrac
list(convergents(127/169))

[(0, 1), (1, 1), (3, 4), (127, 169)]

In [73]:
list(filter(lambda x: x[1]<N,convergents(127/169)))[-1][1]

4

In [63]:
# Encontrando os fatores primos de N
prime_factors = ()

for decimal in decimals:
    if decimal>0:

        # calcula o candidato a r por FC
        print("decimal medido: ", decimal)
        cand_r = list(filter(lambda x: x[1]<N,convergents(decimal/2**m)))[-1][1]
        print("candidato a r: ", cand_r)

        # verifica se r é par
        if cand_r % 2 != 0 or cand_r == 0:
            continue

        # calcula a^r mod N e verifica se r é periodo
        a_r_modulo_N = np.mod(a**cand_r, N)
        if a_r_modulo_N == 1:       
            guesses = gcd(a**int(cand_r/2) + 1, N), gcd(a**int(cand_r/2) - 1, N)
            prime_factors = guesses[0], guesses[1]
            break

print("\n Fatores primos encontrados: ", prime_factors)

decimal medido:  4


ZeroDivisionError: float division by zero

## Parte final

Neste momento, os fatores primos encontrados pelo Algoritmo de Shor serão usados para decriptar uma mensagem encriptada pelo método RSA.

In [ ]:
import numpy as np
import string

def codificacao(x):
    '''
    Arg: string --> converte letras minusculas e espaço em inteiros.
    '''
    if( type(x) == type(1) ):
        alfabeto = list(string.ascii_lowercase) + [' ']
        if( x < len(alfabeto) ):
            return alfabeto[x]
        else:
            return '?'
    if( type(x) == type('s') ):
        alfabeto = list(string.ascii_lowercase) + [' ']
        code = {}
        for inteiro, letra in enumerate(alfabeto):
            code[letra]=inteiro
        return code[x]

# entre com fatores primos encontrados

p = 5
q = 7
n = p*q

# entre com as chaves publica 'e' e chave privada 'd'
e = 5; d=5

M_rsa = [14, 8, 31, 11, 20, 1, 8, 24, 31, 21, 14, 32, 9, 31, 11, 20, 9, 1, 12, 14, 20, 31, 0, 31, 32, 7, 0, 21, 9]

# decodifica a mensagem
mensagem_encriptada = ''
mensagem_decriptada = ''
for j in np.arange(len(M_rsa)):
    mensagem_encriptada = mensagem_encriptada + codificacao( M_rsa[j] )
    mensagem_decriptada = mensagem_decriptada + codificacao( (M_rsa[j]**d)%n )

print("mensagem encriptada: ", mensagem_encriptada)

### Verificando o resultado

In [ ]:
print("mensagem decriptada: ", mensagem_decriptada)